In [9]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [10]:
import numpy as np
%cd /content/drive/MyDrive/
from submit import my_fit
import time as tm
from sklearn.metrics import mean_absolute_error as mae

/content/drive/MyDrive


ModuleNotFoundError: ignored

In [ ]:
X_trn = np.loadtxt( "train_challenges.dat" )
y_trn = np.loadtxt( "train_responses.dat" )
X_tst = np.loadtxt( "dummy_test_challenges.dat" )
y_tst = np.loadtxt( "dummy_test_responses.dat" )
w_gold = np.loadtxt( "dummy_w.dat" )

In [ ]:
# Hard Thresholding operation
# Given a vector and a sparsity level, sparsify the vector
# by setting small magnitude coordinates to 0
def HT( v, k ):
    t = np.zeros_like( v )
    if k < 1:
        return t
    else:
        ind = np.argsort( abs( v ) )[ -k: ]
        t[ ind ] = v[ ind ]
        return t

In [ ]:
def supp_int( x, y, k ):
    x_t = HT( x, k )
    y_t = HT( y, k )
    supp_x = np.where( x_t != 0, 1, 0 )
    supp_y = np.where( y_t != 0, 1, 0 )
    return np.dot( supp_x, supp_y ) / k

In [ ]:
n_trials = 5

t_train = 0
model_err = 0
mae_err = 0
supp_rec = 0

S = 512

In [ ]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  w_hat = HT( my_fit( X_trn, y_trn ), S )
  toc = tm.perf_counter()
  t_train += toc - tic
  model_err += np.linalg.norm( w_hat - w_gold )
  mae_err += mae( y_tst, X_tst @ w_hat )
  supp_rec += supp_int( w_hat, w_gold, S )

In [ ]:
t_train /= n_trials
model_err /= n_trials
mae_err /= n_trials
supp_rec /= n_trials

print( t_train, model_err, mae_err, supp_rec )